# Entrega P1
###### Luis Martín Pérez, Andrea Santana López
---

Se importan las librerías

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

### 1 - Tablero de Ajedrez

In [ ]:
board_size = 800

# Se divide la imagen en una cuadrícula 8 x 8
cell_number = 8
cell_size = int(board_size / cell_number)

# Se inicializa la imagen
chess_board_img = np.zeros((board_size, board_size, 1), dtype=np.uint8)

for row in range(cell_number):
    for column in range(0, cell_number, 2):
        cell_x = row * cell_size
        # En las filas impares se rueda una posición a la derecha
        cell_y = (column + row%2) * cell_size
        # Pintamos las cuadrículas de las columnas pares en blanco
        chess_board_img[cell_x:cell_x + cell_size, cell_y:cell_y + cell_size, 0] = 255

# Mostrar imagen de escala de grises
plt.imshow(chess_board_img, cmap='gray',vmin=0, vmax=255)

### 2 - Imagen estilo ***Mondrian***

In [ ]:
mondrian_size = 1024

# Se divide la imagen en una cuadrícula de 50 x 50
divisions = 50
division_size = int(mondrian_size / divisions)
# El resto de la division se pone como margen para centrar la cuadrícula
margin = int(mondrian_size % divisions / 2)

mondrian_img = np.zeros((mondrian_size, mondrian_size, 3), dtype=np.uint8)

# Estructura de datos para simplificar el diseño
class Rectangle:
    def __init__(self, x, y, end_x, end_y, color):
        self.x = x
        self.y = y
        self.end_x = end_x
        self.end_y = end_y
        self.color = color

rectangles = [
    Rectangle(15,0,50,35,(255,0,0)),
    Rectangle(0,0,14,14,(255,255,255)),
    Rectangle(0,16,14,35,(255,255,255)),
    Rectangle(0,36,14,50,(0,0,255)),
    Rectangle(15,36,45,50,(255,255,255)),
    Rectangle(46,36,50,43,(255,255,255)),
    Rectangle(46,44,50,50,(255,255,0)),
]

# Dentro del bucle se transforman las coordenadas de la cuadrícula a las de la imagen
for r in rectangles:
    start_x = r.x * division_size + margin
    start_y = r.y * division_size + margin
    width = r.end_x - r.x
    height = r.end_y - r.y
    cv2.rectangle(mondrian_img,(start_x ,start_y), (start_x + division_size * width, start_y + division_size * height), r.color,-1)


plt.imshow(mondrian_img)

# Se guarda la imagen en disco para venderla
bgr_img = cv2.cvtColor(mondrian_img, cv2.COLOR_BGR2RGB)
cv2.imwrite('mondrian.jpg', bgr_img)

### 3 - Modificar los planos de color

In [ ]:
vid = cv2.VideoCapture(0)

while(True):
    # fotograma a fotograma
    ret, frame = vid.read()

    if ret:
        # Se redimensiona la captura de la cámara para hacer un collage
        h, w, c = frame.shape
        new_size = (int(w*0.6),int(h/2))
        new_size_rgb = (new_size[1], new_size[0], 3)
        frame = cv2.resize(frame, new_size, cv2.INTER_NEAREST)

        # Separamos canales de color
        b = frame[:,:,0]
        g = frame[:,:,1]
        r = frame[:,:,2]

        # Se inicializan los recuadros correspondientes
        red = np.zeros(new_size_rgb, dtype = np.uint8)
        blue = np.zeros(new_size_rgb, dtype = np.uint8)
        green = np.zeros(new_size_rgb, dtype = np.uint8)

        yellow = np.zeros(new_size_rgb, dtype = np.uint8)
        magenta = np.zeros(new_size_rgb, dtype = np.uint8)
        cyan = np.zeros(new_size_rgb, dtype = np.uint8)

        invert_red = np.zeros(new_size_rgb, dtype = np.uint8)
        invert_green = np.zeros(new_size_rgb, dtype = np.uint8)
        invert_blue = np.zeros(new_size_rgb, dtype = np.uint8)

        creepy_1 = np.zeros(new_size_rgb, dtype = np.uint8)
        creepy_2 = np.zeros(new_size_rgb, dtype = np.uint8)
        creepy_3 = np.zeros(new_size_rgb, dtype = np.uint8)

        # Se separan los planos RGB
        red[:,:,2] = r
        green[:,:,1] = g
        blue[:,:,0] = b

        # Se crean los colores primarios del esquema de color CMY
        yellow[:,:,2] = r
        yellow[:,:,1] = g

        magenta[:,:,2] = r
        magenta[:,:,0] = b

        cyan[:,:,1] = g
        cyan[:,:,0] = b

        # Se invierte cada canal de color
        invert_red[:,:,2] = 255 - r
        invert_red[:,:,1] = g
        invert_red[:,:,0] = b

        invert_green[:,:,2] = r
        invert_green[:,:,1] = 255 - g
        invert_green[:,:,0] = b

        invert_blue[:,:,2] = r
        invert_blue[:,:,1] = g
        invert_blue[:,:,0] = 255 - b

        # Experimentando para obtener imágenes espeluznantes
        creepy_1[:,:,2] = 140 - r
        creepy_1[:,:,1] = b/3
        creepy_1[:,:,0] = 0

        creepy_2[:,:,2] = 60 - b
        creepy_2[:,:,1] = 0
        creepy_2[:,:,0] = g

        creepy_3[:,:,2] = 255 - b
        creepy_3[:,:,1] = 60 - r
        creepy_3[:,:,0] = 30 - g


        # Se crean las filas
        rgb_collage = np.hstack((red, green, blue))
        ymc_collage = np.hstack((yellow, magenta, cyan))
        invert_collage = np.hstack((invert_red, invert_green, invert_blue))
        creepy_collage = np.hstack((creepy_1, creepy_2, creepy_3))

        # Se finaliza el collage agrupando en vertical
        collage = np.vstack((rgb_collage, ymc_collage, invert_collage, creepy_collage))

        cv2.imshow('Color planes', collage)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

vid.release()
cv2.destroyAllWindows()

### 4 - Encontrar el punto más claro y más oscuro de una imagen

In [ ]:


pixels = cv2.imread("CAMPO.jpg", cv2.IMREAD_COLOR)
pixels=cv2.cvtColor(pixels,cv2.COLOR_BGR2RGB)
oscuro,claro = np.array([255, 255, 255], dtype=np.int32),np.array([0, 0, 0], dtype=np.int32)
coordXYC,coordXYO=[0,0],[0,0]

for fila in range(0,pixels.shape[0]):
    for columna in range(0,pixels.shape[1]):
          pixel = pixels[fila, columna].astype(np.int32)
          if np.sum(pixel) > np.sum(claro):
            claro = pixel
            coordXYC = [fila, columna]
          if np.sum(pixel) < np.sum(oscuro):
            oscuro = pixel
            coordXYO = [fila, columna]

pixels = pixels.astype(np.uint8)
print(pixels[coordXYC[0],coordXYC[1]])
print(pixels[coordXYO[0],coordXYO[1]])
cv2.circle(pixels, (coordXYC[1], coordXYC[0]), 40, color=(255, 0, 0), thickness=-1)
cv2.circle(pixels, (coordXYO[1], coordXYO[0]), 40, color=(0, 255, 0), thickness=-1)
plt.imshow(pixels)
plt.show()


### 5 - Propuesta Pop-Art

In [ ]:
minecraft_block_data = []

def average_color(img):
    return tuple(np.mean(img, axis=(0,1)).astype(int))

def calculate_distance(color, r, g, b):
    return np.sqrt((color[0] - r) **2 + (color[1] - g)**2 + (color[2] - b)**2)

def find_nearest_texture(color):
    best_match = minecraft_block_data[0][3]
    min_distance = calculate_distance(color, minecraft_block_data[0][0], minecraft_block_data[0][1], minecraft_block_data[0][2])

    for r, g, b, texture in minecraft_block_data:
        distance = calculate_distance(color, r, g, b)
        if distance < min_distance:
            min_distance = distance
            best_match = texture
    return best_match

# Cargar texturas y clasificarlas según su color más prominente
texture_folder = "textures/"

for filename in os.listdir(texture_folder):
    if filename.endswith(".png"):
        path = os.path.join(texture_folder, filename)
        texture_img = cv2.imread(path)
        minecraft_block_data.append((*average_color(texture_img), texture_img))


In [ ]:
vid = cv2.VideoCapture(0)


# Tamaño de la textura
block_size = 16

while(True):
    # fotograma a fotograma
    ret, frame = vid.read()

    if ret:
        #Dimensiones originales
        h, w, c = frame.shape

        minecraft_rows = int(h/block_size)
        minecraft_columns = int(w/block_size)

        minecraft_img = np.zeros((h, w, 3), dtype=np.uint8)

        # Muestra fotograma resultante
        downscaled_img = cv2.resize(frame, (minecraft_columns, minecraft_rows), interpolation=cv2.INTER_NEAREST)
        downscaled_img[:,:,0] += 5

        for x in range(minecraft_rows):
            for y in range(minecraft_columns):
                minecraft_img[x*block_size:x*block_size + block_size, y*block_size:y*block_size + block_size] = find_nearest_texture(downscaled_img[x][y])

        cv2.imshow('Cam', minecraft_img)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()